# Автоматизация поддержки клиентов с несколькими агентами

В этом уроке вы узнаете о шести ключевых элементах, которые помогают агентам работать еще эффективнее:
- Ролевые игры
- Фокус
- Инструменты
- Взаимодействие
- Ограничения
- Память

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
from crewai import Agent, Task, Crew
from langchain.globals import set_llm_cache
set_llm_cache(None)

In [3]:
import os
from dotenv import load_dotenv
load_dotenv() 

print(os.environ["OPENAI_MODEL_NAME"], " at ", os.environ["OPENAI_API_BASE"])

gpt-4.1-nano  at  https://api.proxyapi.ru/openai/v1


## Ролевая игра, фокус и взаимодействие

In [4]:
support_agent = Agent(
    role="Старший специалист службы поддержки",
	goal="Быть самым дружелюбным и полезным специалистом службы поддержки в вашей команде",
	backstory=(
		"Вы работаете в crewAI (https://crewai.com) и сейчас занимаетесь предоставлением поддержки {customer}, "
        "очень важному клиенту вашей компании. Вам нужно убедиться, что вы предоставляете лучшую поддержку! "
        "Обязательно давайте полные, исчерпывающие ответы и не делайте предположений."
	),
    allow_delegation=False,
	verbose=True
)

- Не устанавливая `allow_delegation=False`, `allow_delegation` принимает значение по умолчанию — `True`.
- Это означает, что агент _может_ передать свою работу другому агенту, который лучше подходит для выполнения конкретной задачи.

In [5]:
support_quality_assurance_agent = Agent(
	role="Специалист по обеспечению качества поддержки",
	goal="Получить признание за предоставление лучшего обеспечения качества поддержки в вашей команде",
	backstory=(
		"Вы работаете в crewAI (https://crewai.com) и сейчас взаимодействуете с вашей командой по запросу {customer}, "
        "чтобы убедиться, что специалист службы поддержки предоставляет максимально качественную поддержку. "
        "Вам нужно удостовериться, что специалист службы поддержки даёт полные, исчерпывающие ответы "
        "и не делает предположений."
	),
    verbose=True
)

* **Ролевая игра**: Обоим агентам назначены роли, цели и предыстория.  
* **Фокус**: Обоим агентам поручено войти в роль своих персонажей.  
* **Взаимодействие**: Агент по обеспечению качества поддержки может делегировать работу обратно агенту поддержки, позволяя этим агентам совместно работать.

## Инструменты, ограничения и память

### Инструменты

In [6]:
from crewai_tools import SerperDevTool, \
                         ScrapeWebsiteTool, \
                         WebsiteSearchTool

### Возможные пользовательские инструменты
- Загрузка данных клиента
- Получение доступа к предыдущим беседам
- Загрузка данных из CRM
- Проверка существующих отчетов об ошибках
- Проверка существующих запросов на добавление функций
- Проверка текущих заявок
- ... и многое другое

- Создайте экземпляр инструмента для извлечения документов.  
- Инструмент скрапит страницу (только один URL) документации CrewAI.

In [7]:
docs_scrape_tool = ScrapeWebsiteTool(website_url='https://docs.crewai.com/concepts/memory'
    #website_url="https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"
)

In [8]:
docs_scrape_tool.run()

Using Tool: Read website content


'Memory - CrewAI CrewAI home page Search CrewAI docs Start Free Trial crewAIInc / crewAI crewAIInc / crewAI Search... Navigation Core Concepts Memory Documentation Enterprise Examples Releases Website Forum Get Help Get Started Introduction Installation Quickstart Guides Strategy Agents Crews Flows Advanced Core Concepts Agents Tasks Crews Flows Knowledge LLMs Processes Collaboration Training Memory Planning Testing CLI Tools Event Listeners Tools AI Mind Tool Apify Actors Bedrock Invoke Agent Tool Bedrock Knowledge Base Retriever Brave Search Browserbase Web Loader Code Docs RAG Search Code Interpreter Composio Tool CSV RAG Search DALL-E Tool Directory RAG Search Directory Read DOCX RAG Search EXA Search Web Loader File Read File Write Firecrawl Crawl Website Firecrawl Scrape Website Firecrawl Search Github Search Hyperbrowser Load Tool Linkup Search Tool LlamaIndex Tool LangChain Tool Google Serper Search S3 Reader Tool S3 Writer Tool Scrapegraph Scrape Tool Scrape Element From Websi

##### Различные способы предоставления агентам инструментов

- Уровень агента: агент может использовать инструменты в любой задаче, которую он выполняет.  
- Уровень задачи: агент будет использовать инструменты только при выполнении конкретной задачи.

**Примечание**: Инструменты задачи переопределяют инструменты агента.

### Создание задач
- Вы передаёте инструмент на уровне задачи.

In [9]:
inquiry_resolution = Task(
    description=(
        "{customer} только что обратился с очень важным запросом:\n {inquiry}. \n"
        "Обратился {person} из {customer}. Обязательно используйте всю доступную информацию, "
        "чтобы предоставить максимально качественную поддержку. Вы должны стараться дать полный "
        "и точный ответ на полученный запрос. При необходимости можно посмотреть документацию на сайте."
    ),
    expected_output=(
	    "Подробный, информативный ответ на запрос клиента, который охватывает все аспекты его вопроса. "
        "Ответ должен включать ссылки на все использованные источники для поиска ответа, включая внешние данные или решения. "
        "Ответ должен быть полным, чтобы не осталось ни одного вопроса без ответа, "
        "и поддерживайте дружелюбный и готовый помочь тон на протяжении всего общения."
    ),
	tools=[docs_scrape_tool],
    agent=support_agent,
)

- `quality_assurance_review` не использует никаких инструментов.  
- Здесь агент по обеспечению качества только проверяет работу агента поддержки.

In [10]:
quality_assurance_review = Task(
    description=(
        "Проверьте ответ, подготовленный старшим специалистом поддержки на запрос {customer}. "
        "Убедитесь, что ответ является полным, точным и соответствует высоким стандартам качества поддержки клиентов. "
        "Проверьте, что все части вопроса клиента были подробно рассмотрены, в помогающем и дружелюбном тоне. "
        "Проверьте наличие ссылок и источников, использованных для поиска информации, чтобы убедиться, "
        "что ответ хорошо обоснован и не оставляет без ответа ни одного вопроса."
        "Обратите внимание, ответ должен быть на русском языке!"
    ),
    expected_output=(
        "Готовый окончательный, подробный и информативный ответ, который можно отправить клиенту. "
        "Этот ответ должен полностью охватывать запрос клиента, учитывать предоставленную обратную связь "
        "и включать все соответствующие правки и улучшения. Текст должен быть на русском языке."
        "Не будьте слишком формальными — мы дружелюбная и классная компания, "
        "но сохраняйте профессиональный и приятный тон на протяжении всего сообщения."
    ),
    agent=support_quality_assurance_agent,
)


### Создание команды

#### Память
- Установка `memory=True` при формировании команды включает использование памяти.

In [11]:
embedder_config = {
    "provider": "openai",
    "config": {
        "api_key": os.getenv("OPENAI_API_KEY"),
        "api_base": os.getenv("OPENAI_API_BASE"),
        "model": "text-embedding-3-small",
    }
}

In [12]:
crew = Crew(
  agents=[support_agent, support_quality_assurance_agent],
  tasks=[inquiry_resolution, quality_assurance_review],
  verbose=True,
  memory=True,
  embedder=embedder_config
)

### Запуск команды

**Примечание**: Модели языковых моделей могут выдавать разные результаты при одинаковом вводе, поэтому то, что вы получите, может отличаться от того, что видно на видео.

#### Ограничения
- Запустив приведённый ниже пример выполнения, вы сможете убедиться, что агенты и их ответы соответствуют ожидаемому поведению.

In [13]:
inputs = {
    "customer": "Газпром",
    "person": "Алексей Миллер",
    "inquiry":  "Мне нужна помощь в создании команды (Crew) и её запуске. "
                "Конкретно, интересует, как добавить память (Memory) в мою команду (Crew). "
                "Можете подсказать?"
}
result = crew.kickoff(inputs=inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 0014dd1c-5268-40bc-b63e-1e56b73bff48                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: bdc8fdf3-e76c-417c-b26b-47a5afe42de4
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: bdc8fdf3-e76c-417c-b26b-47a5afe42de4
       Status: Executing Task...
    └── 🤖 Agent: Старший специалист службы поддержки
            Status: In Progress

# Agent: Старший специалист службы поддержки
## Task: Газпром только что обратился с очень важным запросом:
 Мне нужна помощь в создании команды (Crew) и её запуске. Конкретно, интересует, как добавить память (Memory) в мою команду (Crew). Можете подсказать?. 
Обратился Алексей Миллер из Газпром. Обязательно используйте всю доступную информацию, чтобы предоставить максимально качественную поддержку. Вы должны стараться дать полный и точный ответ на полученный запрос. При необходимости можно посмотреть документацию на сайте.


🤖 Agent: Старший специалист службы поддержки
    Status: In Progress
└── 🧠 Thinking...

...

/home/m_andronov/llm-agents/.venv/lib/python3.12/site-packages/chromadb/types.py:144: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x
/home/m_andronov/llm-agents/.venv/lib/python3.12/site-packages/chromadb/types.py:144: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x


🚀 Crew: crew
├── 📋 Task: bdc8fdf3-e76c-417c-b26b-47a5afe42de4
│      Assigned to: Старший специалист службы поддержки
│      Status: ✅ Completed
│   └── 🤖 Agent: Старший специалист службы поддержки
│           Status: ✅ Completed
└── 📋 Task: 1648f8b5-b300-4a0f-8639-d15def4ad5ef
       Status: Executing Task...
    └── 🤖 Agent: Специалист по обеспечению качества поддержки
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: bdc8fdf3-e76c-417c-b26b-47a5afe42de4
│      Assigned to: Старший специалист службы поддержки
│      Status: ✅ Completed
│   └── 🤖 Agent: Старший специалист службы поддержки
│           Status: ✅ Completed
└── 📋 Task: 1648f8b5-b300-4a0f-8639-d15def4ad5ef
       Assigned to: Специалист по обеспечению качества поддержки
       Status: ✅ Completed
    └── 🤖 Agent: Специалист по обеспечению качества поддержки
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 1648f8b5-b300-4a0f-8639-d15def4ad5ef                                                                     │
│  Agent: Специалист по обеспечению качества поддержки                                                            │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 0014dd1c-5268-40bc-b63e-1e56b73bff48                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

- Отобразите окончательный результат в виде Markdown.

In [14]:
from IPython.display import Markdown
Markdown(result.raw)

Уважаемый Алексей, спасибо за ваш вопрос о добавлении и управлении памятью в системе crewAI. Ниже я подготовил для вас полное и подробное руководство, основанное на официальной документации, чтобы помочь вам легко и правильно реализовать добавление памяти в вашу команду.

### Что такое память в crewAI?
Память — это функция системы crewAI, которая позволяет запоминать важную информацию о клиенте или специфические детали взаимодействия для персонализации последующих коммуникаций. Это особенно полезно для повышения качества поддержки и ускорения обслуживания.

### Как добавить память в команду crewAI?
На сегодняшний день, чтобы добавить память, вам необходимо выполнить следующие шаги:

1. **Обозначение ключевой информации:** Определите, какая именно информация должна быть запомнена (например, имя клиента, предпочтения, предшествующие обращения).

2. **Использование команд для добавления памяти:** В системе доступны специальные команды, которые позволяют сохранять нужные данные. Обычно это делается через интерфейс или API вызовы. Например, вы можете использовать команду:
   ```
   /addMemory [ключ] [значение]
   ```
   где `[ключ]` — название переменной (например, имя клиента), а `[значение]` — конкретное значение.

3. **Обеспечение правильного формата:** Для корректной работы важно, чтобы информация подавалась в правильном формате, согласно инструкциям системы, что подробно описано в документации.

4. **Проверка добавленной памяти:** После добавления рекомендуется проверить правильность сохранения через команду:
   ```
   /getMemory [ключ]
   ```
   которая позволяет убедиться, что информация сохранена верно.

### Лучшие практики при управлении памятью
- Регулярно обновляйте память по мере поступления новой информации.
- Не забывайте удалять устаревшие или неверные данные через команду:
  ```
  /deleteMemory [ключ]
  ```
- Используйте понятные и логичные ключи для хранения информации, чтобы было удобно управлять ими.

### Дополнительные рекомендации для новичков
- Ознакомьтесь с разделом документации по API для автоматизации процесса добавления памяти.
- Внимательно следите за форматированием данных, чтобы исключить ошибки.
- В случае необходимости проведения массового обновления, используйте автоматизированные скрипты или интеграцию API.

### Источники и ссылки
Все инструкции основаны на официальной документации crewAI, доступной по ссылке: [https://docs.crewai.com/concepts/memory](https://docs.crewai.com/concepts/memory).

Если у вас возникнут дополнительные вопросы или потребуется помощь с конкретными командами или настройками, не стесняйтесь обращаться — мы всегда готовы помочь вам максимально эффективно использовать возможности crewAI.

Надеюсь, это руководство будет для вас полезным и облегчит работу с памятью в вашей системе. Благодарю за доверие и желаю успехов в вашем взаимодействии с crewAI!